In [5]:
from PIL import Image
import numpy as np
import skimage.io as io
import tensorflow as tf

In [10]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
  
tfrecords_filename = 'workshop.tfrecords'
writer = tf.python_io.TFRecordWriter(tfrecords_filename)

# Let's collect the real images to later on compare
# to the reconstructed ones
original_images = []
filename_pairs = [("./rose.jpg","rose"),("./daisy.jpg","daisy")]

for img_path, label in filename_pairs:
  img = np.array(Image.open(img_path))
  height = img.shape[0]
  width = img.shape[1]
  img_raw = img.tostring()
  example = tf.train.Example(features=tf.train.Features(feature={
        'height': _int64_feature(height),
        'width': _int64_feature(width),
        'image_raw': _bytes_feature(img_raw),
        'label': _bytes_feature(label)}))
  
  writer.write(example.SerializeToString())

writer.close()


In [12]:
!ls -ltrh workshop.tfrecords

-rw-r--r-- 1 root root 1.5M Mar 14 22:03 workshop.tfrecords


In [13]:
reconstructed_images = []

record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)

for string_record in record_iterator:
    
    example = tf.train.Example()
    example.ParseFromString(string_record)
    
    height = int(example.features.feature['height']
                                 .int64_list
                                 .value[0])
    
    width = int(example.features.feature['width']
                                .int64_list
                                .value[0])
    
    img_string = (example.features.feature['image_raw']
                                  .bytes_list
                                  .value[0])
    
    annotation_string = (example.features.feature['label']
                                .bytes_list
                                .value[0])
    print ("Annotation: " + annotation_string)

Annotation: rose
Annotation: daisy
